In [8]:
import pandas as pd
import xlrd
import numpy as np
import openpyxl
import sys
from openpyxl import Workbook

In [88]:
from openpyxl import load_workbook
wb = load_workbook('Character Classification Perceptron.xlsx')

In [10]:
perceptron = wb['Perceptron']

In [11]:
#Create the table headers  for inputs, weight calculations
perceptron.cell(row = 1, column = 1).value = 'Letter'

#x1-x70 | B = 1
for i in range(1, 71):
    perceptron.cell(row = 1, column = 1 + i).value = 'x' + str(i)    
perceptron.cell(row = 1, column = 72).value = 'B = 1'

#t | yin | y
perceptron.cell(row = 1, column = 73).value = 't'
perceptron.cell(row = 1, column = 74).value = 'yin'
perceptron.cell(row = 1, column = 75).value = 'y'

#Dw1-Dw70 | Db
for i in range(1, 71):
    perceptron.cell(row = 1, column = 75 + i).value = 'Dw' + str(i)
perceptron.cell(row = 1, column = 146).value = 'Db'

#w1-w70 | b
for i in range(1, 71):
    perceptron.cell(row = 1, column = 146 + i).value = 'w' + str(i)
perceptron.cell(row = 1, column = 217).value = 'b'

In [12]:
wb.save('Character Classification Perceptron.xlsx')

In [13]:
A = pd.read_excel('CharacterSet.xlsx', sheet_name = 'A', header=None).to_numpy()
B = pd.read_excel('CharacterSet.xlsx', sheet_name = 'B', header=None).to_numpy()
C = pd.read_excel('CharacterSet.xlsx', sheet_name = 'C', header=None).to_numpy()
D = pd.read_excel('CharacterSet.xlsx', sheet_name = 'D', header=None).to_numpy()
a = pd.read_excel('CharacterSet.xlsx', sheet_name = 'a1', header=None).to_numpy()
b = pd.read_excel('CharacterSet.xlsx', sheet_name = 'b1', header=None).to_numpy()
c = pd.read_excel('CharacterSet.xlsx', sheet_name = 'c1', header=None).to_numpy()
d = pd.read_excel('CharacterSet.xlsx', sheet_name = 'd1', header=None).to_numpy()

In [14]:
data = [A, B, C, D, a, b, c, d]

In [16]:
# Converting graphical data to biploar data
for i in range(0, len(data)):
    columnNo = 2
    for row in range(0, 10):
        for col in range(0, 7):
            if data[i][row - 1][col - 1] == 'x':
                perceptron.cell(row = 2 + i, column = columnNo).value = 1
            else:
                perceptron.cell(row = 2 + i, column = columnNo).value = -1
            columnNo +=1
            
    perceptron.cell(row = 2 + i, column = 72).value = 1
wb.save("Character Classification Perceptron.xlsx")

#Enter the targets manually in t column (column number 73)
#######################################################################################################
#######################################################################################################

In [17]:
df = pd.read_excel("Character Classification Perceptron.xlsx")

In [22]:
df

,Letter,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,w62,w63,w64,w65,w66,w67,w68,w69,w70,b
0,A,1,1,-1,-1,-1,-1,-1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B,-1,1,1,1,1,1,1,-1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C,1,-1,1,1,1,1,1,1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D,-1,1,1,1,1,1,-1,-1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a,1,-1,-1,-1,1,1,1,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,b,-1,-1,-1,1,1,1,1,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,c,1,-1,-1,-1,1,1,1,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,d,1,-1,-1,-1,1,1,1,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# a is the learning rate
a = 0.5

# theta is the threshold for the activation function 
theta = 0

In [80]:
def calculate_weights(row): 

    #Calculate yin = Summation(xi*wi) + b
    if row == 0:
         df['yin'][row] = 0
    else:
        df['yin'][row] = df['b'][row - 1]
        for i in range(1, 71):
            df['yin'][row] = df[str('x' + str(i))][row] * df[str('w' + str(i))][row - 1]

    #Calculate y using the activation function
    if df['yin'][row] > theta:
        df['y'][row] = 1
    elif df['yin'][row] < -theta:
        df['y'][row] = -1
    else:
        df['y'][row] = 0  

#################################################################
# Change in weights
    if df['t'][row] !=  df['y'][row]:
    # CALCULATE CHANGE IN WEIGHTS -> Dw(i) = a * xi * t
        for i in range(1, 71):
            df[str('Dw' + str(i))][row] = a * df[str('x' + str(i))][row] * df['t'][row] 
    #     Db = a * t
        df['Db'][row] = a * df['t'][row]

    
    #######################################
    # UPDATE WEIGHTS -> w(i) = w(i-1) + Dwi
        if row == 0:
            #Assuming initial weights to be 0
            for i in range(1, 71):
                df[str('w' + str(i))][row] = df[str('Dw' + str(i))][row]
            df['b'][row] = df['Db'][row] 
        else:
            for i in range(1, 71):
                df[str('w' + str(i))][row] = df[str('w'+ str(i))][row - 1] + df[str('Dw' + str(i))][row]
            df['b'][row] = df['b'][row - 1] + df['Db'][row]

    else:
    # COPY WEIGHTS FROM LAST ROW-> w(i) = w(i-1)
        if row == 0:
            #Assuming initial weights to be 0
            for i in range(1, 71):
                df[str('w' + str(i))][row] = 0
            df['b'][row] = 0
        else:
            for i in range(1, 71):
                df[str('w' + str(i))][row] = df[str('w'+ str(i))][row - 1]
            df['b'][row] = df['b'][row - 1]

In [81]:
for row in range(0, 8):
    calculate_weights(row)

C:\Users\shiva\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\shiva\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\shiva\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\shiva\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [82]:
df.to_excel('trained_network_perceptron.xlsx')

In [83]:
#Testing the network

In [84]:
upper = pd.read_excel('TestSet.xlsx', sheet_name = 'upper', header=None).to_numpy()
lower = pd.read_excel('TestSet.xlsx', sheet_name = 'lower', header=None).to_numpy()

In [85]:
testData = [upper, lower]

In [87]:
for letter in testData:
    result = float(df['b'][7])
    colNo = 1
    for row in range(0, len(letter)):
        for col in range(0, len(letter[0])):
            coeff = 0
            if letter[row][col] == 'x': 
                coeff = 1
            else: 
                coeff = -1
            result += float(df[str('w' + str(colNo))][7]) * coeff
            colNo += 1
    print(result)

5.5
-12.5
